# Demo: Serenity Derivatives API - Option Pricing

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to price European options on BTC, ETH and SOL.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4

from time import sleep
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from serenity_sdk.renderers.derivatives.widget_tools import OptionChooser, YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import YieldCurveTablePlot, VolatilitySurfaceTablePlot, OptionValuationResultTablePlot
from serenity_sdk.renderers.derivatives.converters import convert_object_dict_to_df
from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion
from serenity_types.pricing.derivatives.options.valuation import DiscountingMethod, MarketDataOverride, OptionValuationRequest, OptionValuation

# set default plot parameters
plt.rcParams['font.size'] = '16'

# create an alias to the api
api = connect_widget.get_api()

# Load samples of pre-defined option instruments

## Prepare a mapping from UUID to native symbol (linked asset native symbol)

In [ ]:
# Let's first remind that our asset ids are represented using uuids
# The mapping between asset id and native symbols are as below:
asset_summaries = api.refdata().get_asset_summaries()
asset_summaries = [{key: value for key, value in summary.items() if key != 'xrefSymbols'} for summary in asset_summaries]
asset_summaries = pd.json_normalize(asset_summaries)[['assetId', 'nativeSymbol']]
asset_summaries = asset_summaries[asset_summaries['nativeSymbol'].isin(['BTC','ETH'])]
asset_summaries

## Predefined option instruments from a csv file
For now, we read from a pre-saved csv file. In future, we plan to support to query predefined option instruments through an API. 
### Read & parse

In [ ]:
# Load sample options in the system & merge with the underlying asset id and symbol
sample_option_data_file = os.path.join('sample_data', 'list_options_20221202.csv')
sample_options = pd.read_csv(sample_option_data_file, parse_dates=['expiry_datetime'])\
    [['linked_asset_id', 'native_symbol', 'asset_id', 'option_type', 'expiry_datetime', 'strike_price','option_style']]\
    .sort_values(['linked_asset_id', 'expiry_datetime', 'strike_price'])
sample_options['expiry_datetime'] = sample_options['expiry_datetime'].dt.tz_localize(None)
sample_options = pd.merge(sample_options, asset_summaries, how='inner', left_on='linked_asset_id', right_on='assetId')
sample_options.drop('assetId', axis=1, inplace=True)
sample_options.rename(columns={'nativeSymbol':'linked_asset_native_symbol'}, inplace=True)

### Peak samples of predefined options

In [ ]:
sample_options.head(3)

## Select the option to use as a base line

In [ ]:
option_chooser = OptionChooser(sample_options)
print('Select an option to play with')
display(option_chooser.get_widget_to_display())

In [ ]:
# Show the details of the option selected
predefined_option_info = option_chooser.get_selected_option()
predefined_option_info

# Option Valuation

Notes: 
* API takes a list of option valuations and identified using 'option_valuation_id' UUID.
For users' convenience, we create a dictionary with human readable keys. 
* The 'values' of the diction are passed to the OptionValuationRequest. 
* Then, we will re-organise the valuation results in terms of the human reable keys

## Helper function for 'compute_option_valuations'

In [ ]:
# Function to run 'compute_option_valuations' and put the valuation results into a helper object 
# to format outputs
def run_compute_option_valuations(the_optvals, as_of_time=None):

    request = OptionValuationRequest(options=[v for v in the_optvals.values()])
    if as_of_time is not None:
        request.as_of_time = as_of_time

    val_results = api.pricer().compute_option_valuations(request)

    # use a helper object for output formatting
    ovr_tp = OptionValuationResultTablePlot(val_results, the_optvals)
    return ovr_tp.results_table

## Define demo option valuations

Use the asset_id (uuid) of the pre-defined option to construct a option valuation object

In [ ]:
# dictionary of option valuations
demo0_optvals = {}
demo0_qty = 10
# use the predefined option
demo0_optvals['predefined'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = demo0_qty, 
    option_asset_id=UUID(predefined_option_info['asset_id']),
    contract_size=1
)

convert_object_dict_to_df(demo0_optvals)


### Run 'compute_option_valuations' - Historical vs Real-time modes

1. Historical valuation mode: With `as_of_time` specified. 
1. Real-time valuation model: With `as_of_time` left empty. 

In [ ]:
# 1. Specify as_of_time and run twice, 5 seconds apart. 
# We should get the same results. 

as_of_time = datetime.utcnow() - timedelta(hours=5)

from time import sleep

a_res = {}
num_calls = 2
sleep_sec = 5
for j in range(num_calls):
    res_table = run_compute_option_valuations(demo0_optvals, as_of_time=as_of_time)
    a_res[f"run_{j}"] = res_table.iloc[:,0]
    if j < num_calls-1:
        print(f'sleeping {sleep_sec} seconds.')
        sleep(sleep_sec)

print('Expected Behaviour: They should return the exactly the same results since "as_of_time" is specified.')
pd.DataFrame(a_res)

In [ ]:
# Leave as_of_time as empty run several times, sleep a few seconds after each run. 

a_res = {}
num_calls = 10
sleep_sec = 1
print(f"Start making {num_calls} calls, {sleep_sec} seconds apart. Please, wait at least {num_calls * sleep_sec} seconds.")
for j in range(num_calls):
    now_ts = datetime.utcnow()
    res_table = run_compute_option_valuations(demo0_optvals)
    a_res[now_ts] = res_table.iloc[:,0]
    if j < num_calls-1:
        sleep(sleep_sec)

print('Expected Behaviour: Results should change over time.')
res_df = pd.DataFrame(a_res)
res_df

In [ ]:
run_labels = res_df.columns

fig = plt.figure(figsize=(14, 8))
ax1=plt.subplot(2,3,1)
plt.plot(run_labels, res_df.loc['pv'], '.-', label='PV')
plt.title('Present Value'), plt.legend()
plt.setp(ax1.get_xticklabels(), visible=False)
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

ax2=plt.subplot(2,3,2, sharex=ax1)
plt.plot(run_labels, res_df.loc['spot_price'], '.-', label='Spot')
plt.title('Spot Price'), plt.legend()
plt.setp(ax2.get_xticklabels(), visible=False)
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

ax3=plt.subplot(2,3,3, sharex=ax1)
plt.plot(run_labels, res_df.loc['forward_price'], '.-', label='Forward Price')
plt.title('Forward Price'), plt.legend()
plt.setp(ax3.get_xticklabels(), visible=False)
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

plt.subplot(2,3,4, sharex=ax1)
plt.plot(run_labels, res_df.loc['projection_rate'], '.-', label='Projection Rate')
plt.plot(run_labels, res_df.loc['discounting_rate'], '.-', label='Discounting Rate')
plt.title('Interest Rates'), plt.legend()
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

plt.subplot(2,3,5, sharex=ax1)
plt.plot(run_labels, res_df.loc['spot_notional'], '.-', label='Spot Notional')
plt.title('Spot Notional'), plt.legend()
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

plt.subplot(2,3,6, sharex=ax1)
plt.plot(run_labels, res_df.loc['delta_ccy'], '.-', label='delta (ccy)')
plt.title('Delta (ccy)'), plt.legend()
plt.ticklabel_format(useOffset=False, style='plain', axis='y')

fig.tight_layout()
fig.autofmt_xdate()
plt.show()

## Option Valuation - Option specified using attributes (strikes, expiry, etc)
We constuct the identical option using the attributes of the pre-defined option

In [ ]:
# Replicate the same option using the attributes of the prefeined option earlier
demo1_optvals = {}
demo1_optvals['predefined'] = demo0_optvals['predefined']
demo1_optvals['predefined_replica'] = OptionValuation(
    option_valuation_id=str(uuid4()),
    qty = demo0_qty, 
    underlier_asset_id=predefined_option_info['linked_asset_id'],
    strike=predefined_option_info['strike_price'],
    expiry=predefined_option_info['expiry_datetime'],
    option_type=predefined_option_info['option_type'],
    option_style=predefined_option_info['option_style'],
    contract_size=1)


# show option vals
convert_object_dict_to_df(demo1_optvals)

In [ ]:
res_table = run_compute_option_valuations(demo1_optvals)
print('Expected Behaviour: Results should be identifical.')
res_table


## With Market Data Overrides

Let's bump market data and see the impact on the present value. 
To this end, 
* Pick an option with the default market data as the 'base' case. 
* Create a collection of new option valuations with market data bumps over a set of bump sizes. 
* Send the base case and the bumped valuations to API to get the PVs back. 

To illustrate the results, 
* Calculate the PV impact of the bumps, i.e. PNL(bumped) = PV(bumped) - PV(base)
* Plot PNLs over bumps.
* Overlay Taylor approximations using Greeks of the base-case valuation. 

### Define a set of option valuations with spot bumps and send them to API

In [ ]:
# spot price 
spot_base = res_table.loc['spot_price'].iloc[0]
bumps_in_perc = [-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0]
# market data bump
spot_bumps = {f'{sb}':spot_base*sb/100 for sb in bumps_in_perc}

# pick the base option to bump
base_optval = demo1_optvals['predefined_replica'].copy()

# dictionary of option valuations with market data bumped
spot_bumps_optvals = {}
spot_bumps_optvals['base'] = base_optval
for sb_key, sb_val in spot_bumps.items():
    optval_this = base_optval.copy()
    optval_this.option_valuation_id=str(uuid4()) # need a unique id 
    optval_this.spot_price_override = MarketDataOverride(additive_bump=sb_val)
    spot_bumps_optvals[f'spot_bump_{sb_key}%'] = optval_this

# show option valuations
convert_object_dict_to_df(spot_bumps_optvals)

In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(spot_bumps_optvals)
res_table

Let's plot
* The Taylor expansion using the delta (and gamma) from the base option should give a good approximation of the spot bump/revals
* At this point, we have some bug to fix (the bug is reported. )

In [ ]:
qty = demo0_qty
result_base = res_table['base']
result_bumps = res_table[[c for c in res_table.columns if c!='base']]

delta_base = result_base['delta']
gamma_base = result_base['gamma']
pv_base = result_base['pv']
sb_vals = np.array(list(spot_bumps.values()))
plt.figure()
plt.plot(sb_vals, result_bumps.loc['pv'] - pv_base, '.-', ms=10, label='bump & pv change')
plt.plot(sb_vals, qty * delta_base * sb_vals, ':', label='1st-order approximation')
plt.plot(sb_vals, qty * (delta_base * sb_vals + 0.5*gamma_base*sb_vals**2), ':', label='2st-order approximation')
plt.plot()
plt.grid()
plt.xlabel('bump'), plt.ylabel('pv change')
plt.legend()
plt.show()

### Vol Bumps

In [ ]:
# vol bmps 
bumps_in_perc = [-20.0, -10.0, -5.0, -2.5, -1.0, 0.0, +1.0, +2.5, +5.0, +10.0, +20.0]
# market data bump
vol_bumps = {f'{sb}':sb/100 for sb in bumps_in_perc}

# pick the base option to bump
base_optval = demo1_optvals['predefined_replica'].copy()

# dictionary of option valuations with market data bumped
vol_bumps_optvals = {}
vol_bumps_optvals['base'] = base_optval
for sb_key, sb_val in vol_bumps.items():
    optval_this = base_optval.copy()
    optval_this.option_valuation_id=str(uuid4()) # need a unique id 
    optval_this.implied_vol_override = MarketDataOverride(additive_bump=sb_val)
    vol_bumps_optvals[f'vol_bump_{sb_key}%'] = optval_this

# show option valuations
convert_object_dict_to_df(vol_bumps_optvals)

In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(vol_bumps_optvals)
res_table

Let's plot. The PnL profile should be well approximated by the first-order Taylor approximation, i.e. 

vega(base case) * (vol bump).

In [ ]:
qty = demo0_qty
result_base = res_table['base']
result_bumps = res_table[[c for c in res_table.columns if c!='base']]

vega_base = result_base['vega']
pv_base = result_base['pv']
sb_vals = np.array(list(vol_bumps.values()))
plt.figure()
plt.plot(sb_vals, result_bumps.loc['pv'] - pv_base, '.-', ms = 10, label='bump & pv change')
plt.plot(sb_vals, qty * vega_base * sb_vals, ':', label='1st-order approximation')
plt.plot()
plt.grid()
plt.xlabel('bump'), plt.ylabel('pv change')
plt.legend()
plt.show()

## Options over Expiries

So far, we considered an option with a specific expiry datetime and a strike. Now, let's consider 'ATM' options over a set of expiries. Here, ATM in the sense that the strike is the same to the spot price. 

In [ ]:
# Expiries

t0 = datetime.utcnow()
expiry_times = {d:t0 + timedelta(d) for d in [7, 14, 30, 60, 90, 180, 360, 720]}


base_optval = demo1_optvals['predefined_replica'].copy()
base_optval.strike = spot_base # spot ATM options
expiries_optvals = {}
for d, ex in expiry_times.items():
    optval_this = base_optval.copy()
    optval_this.option_valuation_id = str(uuid4())
    optval_this.expiry = ex
    expiries_optvals[f"{d}-days"] = optval_this

# show option valuations
convert_object_dict_to_df(expiries_optvals)


In [ ]:
# Run 'compute_option_valuations'
res_table = run_compute_option_valuations(expiries_optvals)
res_table

Let's plot some fields from the valuation results over expiries

In [ ]:
# plotting
ex_ts = np.array(list(expiry_times.values()))
tt_ex = np.array(list(expiry_times.keys()))/365

fig = plt.figure(figsize=(14, 6))
ax1 = plt.subplot(1,3,1)
plt.plot(ex_ts, res_table.loc['pv'], '.-', label='pv')
plt.grid()
plt.xlabel('expiry datetime'), plt.ylabel('pv')
plt.legend()
plt.xticks(rotation = 90)

ax2 = plt.subplot(1,3,2)
p = res_table.loc['projection_rate'].to_numpy(dtype=np.float64)
plt.plot(ex_ts, p, '.-', label='projection rate')
plt.grid()
plt.xlabel('expiry datetime'), plt.ylabel('rate')
plt.legend()
plt.xticks(rotation = 90)

ax2 = plt.subplot(1,3,3)
sigma = res_table.loc['iv'].to_numpy(dtype=np.float64)
plt.plot(ex_ts, sigma, '.-', label='volatility')
plt.grid()
plt.xlabel('expiry datetime'), plt.ylabel('volatility')
plt.legend()
plt.xticks(rotation = 90)

fig.tight_layout()
plt.show()

# Option Valuation Request overrides

In [ ]:
# TO BE CONTINUED

# END